In [1]:
import radiomics
from radiomics import featureextractor
import glob
import os
import pandas as pd
from pandas import DataFrame
import csv

In [2]:
paramPath = 'Params_tol_0_0001.yaml'

In [3]:
data_path = 'Data/CT/' #path immagine CT
mask_path = 'Data/Masks_Lung/'  #path maschere lung     
images_fnames = glob.glob(os.path.join(data_path,'*.nii.gz'))
masks_fnames = glob.glob(os.path.join(mask_path,'*_Lung.nii.gz'))
images_fnames.sort(key=lambda x: int(''.join(filter(str.isdigit, x))))   #ordinare immagini e maschere in base al nome, in questo caso è un numero, ma riadattare al caso specifico
masks_fnames.sort(key=lambda x: int(''.join(filter(str.isdigit, x)))) 

In [4]:
print(images_fnames)

['Data/CT/volume-covid19-A-0254.nii.gz', 'Data/CT/volume-covid19-A-0505.nii.gz']


In [5]:
print(masks_fnames)

['Data/Masks_Lung/volume-covid19-A-0254_Lung.nii.gz', 'Data/Masks_Lung/volume-covid19-A-0505_Lung.nii.gz']


In [6]:
extractor = featureextractor.RadiomicsFeatureExtractor(paramPath)   #leggo il file di configurazione dei parametri e verifico di usare quelli giusti
verbose = True
if verbose:
    print('Extraction parameters:\n\t', extractor.settings)
    print('Enabled filters:\n\t', extractor.enabledImagetypes)
    print('Enabled features:\n\t', extractor.enabledFeatures)


Extraction parameters:
	 {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 25, 'weightingNorm': None, 'geometryTolerance': 0.0001}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'shape': None, 'firstorder': [], 'glcm': ['Autocorrelation', 'JointAverage', 'ClusterProminence', 'ClusterShade', 'ClusterTendency', 'Contrast', 'Correlation', 'DifferenceAverage', 'DifferenceEntropy', 'DifferenceVariance', 'JointEnergy', 'JointEntropy', 'Imc1', 'Imc2', 'Idm', 'Idmn', 'Id', 'Idn', 'InverseVariance', 'MaximumProbability', 'SumEntropy', 'SumSquares'], 'glrlm': None, 'glszm': None, 'gldm': None}


In [7]:
#feature extraction from Lungs
def_features = [{},{}]
restotal1=[0 for i in range(len(images_fnames))]
restotal=[]

cwdfeatures = 'Features/'
for i in range(len(images_fnames)):
    imageName = images_fnames[i]
    maskNamepre = masks_fnames[i]
   
    print("Calculating features from lung masks")
    print(imageName)
    print(maskNamepre)
    result = extractor.execute(imageName, maskNamepre)
    result['image_ID'] = os.path.basename(str(imageName))
    feat = []
    valore = []
    restotal.append(result)
    for key, value in result.items():
        feat.append(key)
        valore.append(value)
    basename = os.path.basename(imageName)  
    name_no_ext = basename.replace('.nii.gz', '')  
    numeric_id = name_no_ext.split('-')[-1]  
    def_features = {'features':feat,'ID_'+numeric_id:valore}
    resframe=pd.DataFrame(def_features)
    restotal1[i]=pd.DataFrame(resframe)
    resframe.to_csv(os.path.join(cwdfeatures,'features_'+numeric_id+'.csv'), index = False)  
    print('Computed ',i, '\n')
    

Calculating features from lung masks
Data/CT/volume-covid19-A-0254.nii.gz
Data/Masks_Lung/volume-covid19-A-0254_Lung.nii.gz
Computed  0 

Calculating features from lung masks
Data/CT/volume-covid19-A-0505.nii.gz
Data/Masks_Lung/volume-covid19-A-0505_Lung.nii.gz
Computed  1 



In [9]:
csv_columns =  list(restotal[0].keys())

In [10]:
csv_file = "Total_features.csv"

with open(csv_file, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns, extrasaction='ignore' )
    writer.writeheader()
    for data in restotal:
        writer.writerow(data)

In [11]:
pd.set_option('display.max_columns', None)
Lung_features = pd.read_csv("Total_features.csv")
Lung_features

,diagnostics_Versions_PyRadiomics,diagnostics_Versions_Numpy,diagnostics_Versions_SimpleITK,diagnostics_Versions_PyWavelet,diagnostics_Versions_Python,diagnostics_Configuration_Settings,diagnostics_Configuration_EnabledImageTypes,diagnostics_Image-original_Hash,diagnostics_Image-original_Dimensionality,diagnostics_Image-original_Spacing,diagnostics_Image-original_Size,diagnostics_Image-original_Mean,diagnostics_Image-original_Minimum,diagnostics_Image-original_Maximum,diagnostics_Mask-original_Hash,diagnostics_Mask-original_Spacing,diagnostics_Mask-original_Size,diagnostics_Mask-original_BoundingBox,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,diagnostics_Mask-original_CenterOfMassIndex,diagnostics_Mask-original_CenterOfMass,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,original_shape_Sphericity,original_shape_SurfaceArea,original_shape_SurfaceVolumeRatio,original_shape_VoxelVolume,original_firstorder_10Percentile,original_firstorder_90Percentile,original_firstorder_Energy,original_firstorder_Entropy,original_firstorder_InterquartileRange,original_firstorder_Kurtosis,original_firstorder_Maximum,original_firstorder_MeanAbsoluteDeviation,original_firstorder_Mean,original_firstorder_Median,original_firstorder_Minimum,original_firstorder_Range,original_firstorder_RobustMeanAbsoluteDeviation,original_firstorder_RootMeanSquared,original_firstorder_Skewness,original_firstorder_TotalEnergy,original_firstorder_Uniformity,original_firstorder_Variance,original_glcm_Autocorrelation,original_glcm_JointAverage,original_glcm_ClusterProminence,original_glcm_ClusterShade,original_glcm_ClusterTendency,original_glcm_Contrast,original_glcm_Correlation,original_glcm_DifferenceAverage,original_glcm_DifferenceEntropy,original_glcm_DifferenceVariance,original_glcm_JointEnergy,original_glcm_JointEntropy,original_glcm_Imc1,original_glcm_Imc2,original_glcm_Idm,original_glcm_Idmn,original_glcm_Id,original_glcm_Idn,original_glcm_InverseVariance,original_glcm_MaximumProbability,original_glcm_SumEntropy,original_glcm_SumSquares,original_glrlm_GrayLevelNonUniformity,original_glrlm_GrayLevelNonUniformityNormalized,original_glrlm_GrayLevelVariance,original_glrlm_HighGrayLevelRunEmphasis,original_glrlm_LongRunEmphasis,original_glrlm_LongRunHighGrayLevelEmphasis,original_glrlm_LongRunLowGrayLevelEmphasis,original_glrlm_LowGrayLevelRunEmphasis,original_glrlm_RunEntropy,original_glrlm_RunLengthNonUniformity,original_glrlm_RunLengthNonUniformityNormalized,original_glrlm_RunPercentage,original_glrlm_RunVariance,original_glrlm_ShortRunEmphasis,original_glrlm_ShortRunHighGrayLevelEmphasis,original_glrlm_ShortRunLowGrayLevelEmphasis,original_glszm_GrayLevelNonUniformity,original_glszm_GrayLevelNonUniformityNormalized,original_glszm_GrayLevelVariance,original_glszm_HighGrayLevelZoneEmphasis,original_glszm_LargeAreaEmphasis,original_glszm_LargeAreaHighGrayLevelEmphasis,original_glszm_LargeAreaLowGrayLevelEmphasis,original_glszm_LowGrayLevelZoneEmphasis,original_glszm_SizeZoneNonUniformity,original_glszm_SizeZoneNonUniformityNormalized,original_glszm_SmallAreaEmphasis,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_gldm_DependenceEntropy,original_gldm_DependenceNonUniformity,original_gldm_DependenceNonUniformityNormalized,original_gldm_DependenceVariance,original_gldm_GrayLevelNonUniformity,original_gldm_GrayLevelVariance,original_gldm_HighGrayLevelEmphasis,original_gldm_LargeDependenceEmphasis,original_gldm_LargeDependenceHighGrayLevelEmphasis,original_gldm_LargeDependenceLowGrayLevelEmphasis,original_gldm_LowGrayLevelEmphasis,original_gldm_SmallD

In [12]:
pd.set_option('display.max_columns', None)
Lung_features.columns

Index(['diagnostics_Versions_PyRadiomics', 'diagnostics_Versions_Numpy',
       'diagnostics_Versions_SimpleITK', 'diagnostics_Versions_PyWavelet',
       'diagnostics_Versions_Python', 'diagnostics_Configuration_Settings',
       'diagnostics_Configuration_EnabledImageTypes',
       'diagnostics_Image-original_Hash',
       'diagnostics_Image-original_Dimensionality',
       'diagnostics_Image-original_Spacing',
       ...
       'original_gldm_GrayLevelVariance',
       'original_gldm_HighGrayLevelEmphasis',
       'original_gldm_LargeDependenceEmphasis',
       'original_gldm_LargeDependenceHighGrayLevelEmphasis',
       'original_gldm_LargeDependenceLowGrayLevelEmphasis',
       'original_gldm_LowGrayLevelEmphasis',
       'original_gldm_SmallDependenceEmphasis',
       'original_gldm_SmallDependenceHighGrayLevelEmphasis',
       'original_gldm_SmallDependenceLowGrayLevelEmphasis', 'image_ID'],
      dtype='object', length=123)